# Use AutoEncoding with force_torque_sensor

--------------------------------------------------------------------

Datasets paper: https://arxiv.org/pdf/1807.06749.pdf

Download: https://ibm.ent.box.com/s/vw4y576xlz6ujblpl3gz9c5ttu51qc18

Modified by github : "Variational Autoencoder & Conditional Variational Autoenoder on MNIST"

In [1]:
import numpy as np 

sides_3 = np.load('../data/force_torque_sensor/Dataset/3_sides/Data/data.npy')
# sides_4 = np.load('../data/force_torque_sensor/Dataset/4_sides/Data/data.npy')
# sides_5 = np.load('../data/force_torque_sensor/Dataset/5_sides/Data/data.npy')
# sides_6 = np.load('../data/force_torque_sensor/Dataset/6_sides/Data/data.npy')
# sides_200 = np.load('../data/force_torque_sensor/Dataset/200_sides/Data/data.npy')


import numpy as np 
from collections import namedtuple
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
import plotly


/home/jim/anaconda2/envs/clustering/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from collections import defaultdict
import torch.nn.functional as F


from models import VAE

import numpy as np


print(torch.cuda.is_available() )

True


In [3]:
import torch
vae = torch.load('vae_1epoch_16_16.pkl').cuda()
print(vae)

# vae.encoder()

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=12, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=8, bias=True)
    (linear_means): Linear(in_features=8, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=8, out_features=2, bias=True)
  )
  (decoder): Decoder(
    (fc3): Linear(in_features=2, out_features=8, bias=True)
    (fc4): Linear(in_features=8, out_features=32, bias=True)
    (fc5): Linear(in_features=32, out_features=12, bias=True)
  )
)


In [4]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std

def un_normalize(normalized_data, input_data):
    mu = np.mean(input_data,axis=0)
    std = np.std(input_data,axis=0)
    return normalized_data*std+mu

In [5]:
test_data = feature_normalize(sides_3[0])
print(test_data)

[-0.01539689 -0.01533028  0.0070817  -0.22483476 -2.18064528  0.13496538
 -0.08142799 -0.13187814 -0.19902119  0.03084667  2.66661449  0.00902629]


In [6]:
import torch.utils.data

test_loader = torch.from_numpy(test_data).cuda().float()
print(test_loader)

tensor([-0.0154, -0.0153,  0.0071, -0.2248, -2.1806,  0.1350, -0.0814, -0.1319,
        -0.1990,  0.0308,  2.6666,  0.0090], device='cuda:0')


In [7]:

latent = vae.encoder(test_loader)
print(latent)
output = vae.decoder(latent[0]).detach().cpu().numpy()

# generate_data = output[0].detach().cpu().numpy()
# print (len(output))
print (output)

(tensor([-0.0011, -0.0003], device='cuda:0', grad_fn=<AddBackward0>), tensor([1.2278e-05, 2.0234e-05], device='cuda:0', grad_fn=<AddBackward0>))
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
slides_3_generate = []
latent_space = []

for test in sides_3:
    test_loader = torch.from_numpy(test).cuda().float()
    latent = vae.encoder(test_loader)
    output = vae.decoder(latent[0]).detach().cpu().numpy()
    
    slides_3_generate.append(output)
    latent_space.append(latent[0].detach().cpu().numpy())
    

print (len(slides_3_generate))

np.save('slides_3_generate.npy',slides_3_generate)
np.save('latent_space.npy',latent_space)


KeyboardInterrupt: 

In [ ]:
print(type(slides_3_generate))

In [ ]:
slides_3_generate = np.array(slides_3_generate)
print(slides_3_generate[1000])

In [ ]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

cmap=plt.cm.get_cmap(plt.cm.viridis,143)


# plt.rcParams['font.size'] = 11.
# plt.rcParams['font.family'] = 'Comic Sans MS'
# plt.rcParams['axes.labelsize'] = 15.
# plt.rcParams['xtick.labelsize'] = 10.
# plt.rcParams['ytick.labelsize'] = 10.

plt.figure(figsize=(30,10))



sides = slides_3_generate
print(len(sides))

Force_x = sides[:,0] 
Force_y = sides[:,1]
Force_z = sides[:,2]
Moment_x = sides[:,3]
Moment_y = sides[:,4]
Moment_z = sides[:,5]
Peg_Position_x = sides[:,6]
Peg_Position_y = sides[:,7]
Peg_Position_z = sides[:,8]
Angle = sides[:,9]
Time = sides[:,10]
Counter = sides[:,11]


plt.plot(Time, Force_x, marker='s', linestyle='-', markersize=2, linewidth=1, label='Force_x')
plt.plot(Time, Force_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_y')
plt.plot(Time, Force_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_z')

plt.plot(Time, Moment_x, marker='o', linestyle='-', label='Moment_x')
plt.plot(Time, Moment_y, marker='o', linestyle='-', label='Moment_y')
plt.plot(Time, Moment_z, marker='o', linestyle='-', label='Moment_z')

plt.plot(Time, Peg_Position_x, marker='o', linestyle='-', label='Peg_Position_x')
plt.plot(Time, Peg_Position_y, marker='o', linestyle='-', label='Peg_Position_y')
plt.plot(Time, Peg_Position_z, marker='o', linestyle='-', label='Peg_Position_z')

plt.plot(Time, Angle, marker='o', linestyle='-', label='Angle')
plt.plot(Time, Counter, marker='o', linestyle='-', label='Counter')


plt.xlabel('Times')
plt.ylabel('Signals')
plt.title('3 Sileds generated force torque signals')
plt.legend(loc='lower right')
plt.show()

In [ ]:
print(len(Time))